In [162]:
import pandas as pd
import numpy as np
from pathlib import Path

RAW = Path("../data/raw")
PROC = Path("../data/processed")

RAW.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)

RAW, PROC


(PosixPath('../data/raw'), PosixPath('../data/processed'))

In [193]:
owid_url = "https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv"
owid_path = RAW / "owid-co2.csv"

df_co2 = pd.read_csv(owid_url)
df_co2.to_csv(owid_path, index=False)

print("Saved:", owid_path)
print("Rows:", len(df_co2))

df_co2.head()

Saved: ../data/raw/owid-co2.csv
Rows: 50411


,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1750,AFG,2802560.0,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1751,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1752,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1753,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1754,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
co2 = df_co2[
    ["country", "iso_code", "year", "co2", "co2_per_capita"]
].rename(columns={"iso_code": "iso3"})

co2 = co2[
    (co2["iso3"].str.len() == 3) &
    (co2["year"].between(2009, 2023))
].copy()

co2.head()


,country,iso3,year,co2,co2_per_capita
259,Afghanistan,AFG,2009,6.388,0.233
260,Afghanistan,AFG,2010,8.365,0.296
261,Afghanistan,AFG,2011,11.410,0.389
262,Afghanistan,AFG,2012,9.731,0.318
263,Afghanistan,AFG,2013,8.891,0.281


In [189]:
co2 = co2[
    (co2["iso3"].str.len() == 3) &
    (co2["year"].between(2009, 2023))
].copy()
co2.shape

(3052, 5)

In [178]:
oecd_iso3 = [
    "AUS","AUT","BEL","CAN","CHL","COL","CRI","CZE","DNK","EST","FIN",
    "FRA","DEU","GRC","HUN","ISL","IRL","ISR","ITA","JPN","KOR","LVA",
    "LTU","LUX","MEX","NLD","NZL","NOR","POL","PRT","SVK","SVN","ESP",
    "SWE","CHE","TUR","GBR","USA"
]

oecd = pd.DataFrame({"iso3": oecd_iso3})
oecd


,iso3
0,AUS
1,AUT
2,BEL
3,CAN
4,CHL
5,COL
6,CRI
7,CZE
8,DNK
9,EST


In [197]:
co2_oecd = (
    co2.merge(oecd, on="iso3", how="inner")
       .sort_values(["iso3", "year"])
       .reset_index(drop=True)
)
co2_oecd["delta_co2"] = co2_oecd.groupby("iso3")["co2"].diff()


In [198]:
wdi_url = (
    "https://api.worldbank.org/v2/en/indicator/"
    "NY.GDP.MKTP.KD.ZG?downloadformat=csv"
)

gdp_zip = RAW / "wdi_gdp_growth.zip"
gdp_zip

PosixPath('../data/raw/wdi_gdp_growth.zip')

In [199]:
import zipfile
import requests

r = requests.get(wdi_url)
gdp_zip.write_bytes(r.content)

with zipfile.ZipFile(gdp_zip, "r") as z:
    z.extractall(RAW)

RAW

PosixPath('../data/raw')

In [200]:
gdp_file = [f for f in RAW.glob("API_NY.GDP.MKTP.KD.ZG*.csv")][0]
gdp_raw = pd.read_csv(gdp_file, skiprows=4)

# Identify year columns (they are numeric strings)
year_cols = [c for c in gdp_raw.columns if c.isdigit()]

gdp = (
    gdp_raw
    .rename(columns={"Country Code": "iso3"})
    .melt(
        id_vars=["iso3"],
        value_vars=year_cols,
        var_name="year",
        value_name="gdp_growth"
    )
)

gdp["year"] = gdp["year"].astype(int)
gdp.head()


,iso3,year,gdp_growth
0,ABW,1960,NaN
1,AFE,1960,NaN
2,AFG,1960,NaN
3,AFW,1960,NaN
4,AGO,1960,NaN


In [201]:
gdp_oecd = gdp.merge(oecd, on="iso3", how="inner")
gdp_oecd = gdp_oecd[gdp_oecd["year"].between(2010, 2023)]
gdp_oecd.shape

(532, 3)

In [206]:
panel = (
    co2_oecd
    .merge(gdp_oecd, on=["iso3", "year"], how="inner")
    .sort_values(["iso3", "year"])
    .reset_index(drop=True)
)

panel = panel[panel["year"].between(2010, 2023)].copy()

panel.head()


,country,iso3,year,co2,co2_per_capita,delta_co2,gdp_growth
0,Australia,AUS,2010,405.026,18.293,-1.951,2.219980
1,Australia,AUS,2011,403.874,17.966,-1.152,2.407080
2,Australia,AUS,2012,405.005,17.722,1.131,3.916709
3,Australia,AUS,2013,397.850,17.122,-7.155,2.602193
4,Australia,AUS,2014,392.610,16.639,-5.240,2.595284


In [207]:
panel["decoupled"] = np.where(
    panel["gdp_growth"].notna() & panel["delta_co2"].notna(),
    ((panel["gdp_growth"] > 0) & (panel["delta_co2"] < 0)).astype(int),
    np.nan
)
panel[["iso3","year","gdp_growth","co2","delta_co2","decoupled"]].head(10)

,iso3,year,gdp_growth,co2,delta_co2,decoupled
0,AUS,2010,2.219980,405.026,-1.951,1.0
1,AUS,2011,2.407080,403.874,-1.152,1.0
2,AUS,2012,3.916709,405.005,1.131,0.0
3,AUS,2013,2.602193,397.850,-7.155,1.0
4,AUS,2014,2.595284,392.610,-5.240,1.0
5,AUS,2015,2.190169,400.589,7.979,0.0
6,AUS,2016,2.759386,409.666,9.077,0.0
7,AUS,2017,2.279592,413.408,3.742,0.0
8,AUS,2018,2.878470,414.781,1.373,0.0
9,AUS,2019,2.171545,414.925,0.144,0.0


In [211]:
panel_final = panel.dropna(subset=["gdp_growth", "co2", "delta_co2", "decoupled"]).copy()
panel_final.shape

(532, 8)

In [209]:
out_path = PROC / "oecd_panel_2010_2023.csv"
panel_final.to_csv(out_path, index=False)

out_path

PosixPath('../data/processed/oecd_panel_2010_2023.csv')